# 1141_社會資料分析 作業三
學生：劉晏成

## 題目

以「2018台北大學社會系三鶯調查」（brilliant_working.sav）檔案回答以下問題：
1. 受訪者的平均教育年數（eduy）是否為12年？
2. 不同性別（a1）受訪者的教育程度（a8）是否有顯著差異？
3. 受訪者的教育程度（a8）和其配偶的教育程度（a10）有無顯著差異？

In [8]:
from load import load_sav

df = load_sav("../data/brilliant_working.sav")

In [11]:
variable_value_labels = df.attrs['variable_value_labels']
column_names = df.attrs['column_names']
column_names_to_labels = df.attrs['column_names_to_labels']

In [7]:
target_columns = ['eduy', 'a1', 'a8', 'a10']
df[target_columns]

,eduy,a1,a8,a10
0,16.0,2.0,18.0,18.0
1,16.0,1.0,18.0,8.0
2,16.0,2.0,18.0,NaN
3,12.0,1.0,8.0,NaN
4,12.0,2.0,6.0,19.0
...,...,...,...,...
863,6.0,2.0,3.0,3.0
864,12.0,2.0,8.0,8.0
865,6.0,2.0,3.0,4.0
866,12.0,1.0,8.0,NaN


In [14]:
for key in target_columns:
    print(f"{key}: {variable_value_labels.get(key)}")

eduy: None
a1: {1.0: '男', 2.0: '女'}
a8: {1.0: '無', 2.0: '自修', 3.0: '小學肄業', 4.0: '國(初)中', 5.0: '初職', 6.0: '高中普通科', 7.0: '高中職業科', 8.0: '高職', 9.0: '士官學校', 10.0: '五專', 11.0: '二專', 12.0: '三專', 13.0: '軍警專修班', 14.0: '軍警專科班', 15.0: '空中行專', 16.0: '空中大學', 17.0: '軍警官學校', 18.0: '技術學院、科大', 19.0: '大學', 20.0: '碩士', 21.0: '博士', 22.0: '其他', 997.0: '拒答', 998.0: '漏答', 999.0: '不適用'}
a10: {1.0: '無(不識字)', 2.0: '自修(識字、私塾)', 3.0: '小學肄業', 4.0: '國(初)中', 5.0: '初職', 6.0: '高中普通科', 7.0: '高中職業科', 8.0: '高職', 9.0: '士官學校', 10.0: '五專', 11.0: '二專', 12.0: '三專', 13.0: '軍警專修班', 14.0: '軍警專科班', 15.0: '空中行專', 16.0: '空中大學', 17.0: '軍警官學校', 18.0: '技術學院、科大', 19.0: '大學', 20.0: '碩士', 21.0: '博士', 22.0: '其他', 997.0: '拒答', 998.0: '漏答', 999.0: '不適用'}


## Q1: 受訪者的平均教育年數（eduy）是否為12年？

觀察數值分布

In [18]:
df_1.value_counts().sort_index()

eduy
0.0       4
3.0       1
6.0      55
8.0       1
9.0      91
12.0    290
13.0      1
14.0     95
15.0      9
16.0     64
18.0    253
20.0      4
Name: count, dtype: int64

由於教育年數為Ratio scale ，並且數值0 有其意義，故不做recoding 也不排除數值

In [ ]:
以樣本做母體平均數檢定
H0: mean = 12
H1: mean != 12

並且以p value < 0.05 為檢定

In [23]:
from scipy import stats

mu_0 = 12
t_statistic, p_value = stats.ttest_1samp(df_1['eduy'], mu_0)
print(f"t: {t_statistic}, p: {p_value}")

t: 12.310245460523017, p: 3.281672369615777e-32


### Answer

t-test 結果為t: 12.31, p value: 3.28e-32
是故